In [ ]:
user = ""
password= ""
name = ""
url = ""
target_date =""
target_zone=[]
price=[]

In [ ]:
from selenium.webdriver.common.by import By
from seleniumbase import Driver
import re
import time

In [6]:
def select_option_containing_text(driver, select_selector, partial_text):
    # Find all options inside the select
    options = driver.find_elements(f"{select_selector} option")
    for option in options:
        if partial_text in option.text:
            option.click()
            print("date selected")
            return
    raise Exception(f"No option containing '{partial_text}' found")

In [42]:
def get_zones_without_zero(driver) -> list:
    no_zero = []
    # change selector to match your table container if needed
    rows = driver.find_elements("tr[onclick^='gonextstep']")
    for r in rows:
        onclick = r.get_attribute("onclick") or ""
        # extract letter from onclick: gonextstep('fixed.php','A',event);
        m = re.search(r"gonextstep\([^,]+,'([^']+)'", onclick)
        letter = m.group(1) if m else None
        # try to get the numeric text from the second td / link
        try:
            num_el = el = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)")
            num_text = num_el.get_attribute("innerText").strip()
        except Exception:
            num_text = ""

        # normalize (remove quotes, whitespace)
        num_text = num_text.replace('"', '').strip()

        # keep if there is at least one digit and no '0' digit present
        if num_text != "0" and num_text != "Not Available":
            no_zero.append(letter)
    print(f"no zone: {no_zero}")
    return no_zero

In [ ]:
driver = Driver(uc=True, headless=False)

# open page
driver.uc_open_with_reconnect(url, reconnect_time=4)
driver.maximize_window()
print("page opened")

# attempt to login and click CAPTCHA
driver.type("input[name='username']", user)
driver.type("input[name='password']", password)
driver.uc_gui_click_captcha()
print("cloudflare check")
driver.click("button.btn-red.btn-signin")
print("login complete")

driver.wait_for_element_visible("#rdId", timeout=10)
select_option_containing_text(driver, "#rdId", target_date)
driver.click("#popup-avail")
driver.wait_for_element_visible("tr[onclick^='gonextstep']", timeout=10)
no_zone=get_zones_without_zero(driver)
available_zone = [zone for zone in target_zone if zone in no_zone]
print(f"available_zone: {available_zone}")
driver.click(f"tr[onclick*=\"'{available_zone[0]}'\"]")

# get all available seats
available_seats = driver.find_elements(
    "table#tableseats td:not(.not-available):not(.skiprow):not(.headrow)"
)

print("found", len(available_seats), "available seats")

# click first available seat
if available_seats:
    available_seats[0].click()

driver.click("#booknow")
driver.type("#txt_firstname_0", name)
driver.click("#btn_regnow")

page opened
cloudflare check
login complete
date selected
no zone: ['A', 'B', 'T', 'U', 'STDA', 'STDB']
available_zone: ['U', 'T']
found 230 available seats
